In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import cv2
import torch
import torch
import requests
import PIL
from PIL import Image, ImageDraw, ImageFont
from PIL import Image
from PIL import Image
import requests
import io
import math
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
!pip install git+https://github.com/cocodataset/panopticapi.git

  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-req-build-2dh03yyo
  Running command git clone -q https://github.com/cocodataset/panopticapi.git /tmp/pip-req-build-2dh03yyo
  Created wheel for panopticapi: filename=panopticapi-0.1-py3-none-any.whl size=8306 sha256=d821dc323602616ea2eeeda765061416a9a33f9b507cbd1d29601d7d714b5590
  Stored in directory: /tmp/pip-ephem-wheel-cache-n40us357/wheels/ad/89/b8/b66cce9246af3d71d65d72c85ab993fd28e7578e1b0ed197f1
Successfully built panopticapi


In [ ]:
import panopticapi
from panopticapi.utils import id2rgb, rgb2id
from panopticapi.utils import rgb2id, id2rgb

In [ ]:
import torchvision.transforms as T

# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize((800,600)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

In [ ]:
def filter_bboxes_from_outputs(outputs,
                               threshold=0.7):
  
  # keep only predictions with confidence above threshold
  probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
  keep = probas.max(-1).values > threshold

  probas_to_keep = probas[keep]

  # convert boxes from [0; 1] to image scales
  bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
  
  return probas_to_keep, bboxes_scaled

In [ ]:
import itertools
import seaborn as sns
palette = itertools.cycle(sns.color_palette())



In [ ]:
def plot_finetuned_results(pil_img, seg_img, prob=None, boxes=None,subplot_counter=1):
    #plt.figure(figsize=(8,8))
    plt.subplot(1, 4, 3)
    plt.imshow(pil_img)
    #plt.axis('off')
    
    ax = plt.gca()
    colors = COLORS * 100
    if prob is not None and boxes is not None:
      for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
          ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                    fill=False, color=c, linewidth=3))
          cl = p.argmax()
          text = f'{finetuned_classes[cl]}: {p[cl]:0.2f}'
          ax.text(xmin, ymin, text, fontsize=15,
                  bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    #plt.figure(figsize=(10,3))
    #plt.show()
    

    plt.subplot(1, 4, 4)
    #plt.figure(figsize=(5,4))
    #seg_img = seg_img.resize((256, 256))
    plt.imshow(seg_img)
    plt.axis('off')
    #plt.show()

In [ ]:
def run_worflow(my_image, my_model,counter):
  # mean-std normalize the input image (batch-size: 1)
  img = transform(my_image).unsqueeze(0)

  # propagate through the model
  outputs = my_model(img)
  palette = itertools.cycle(sns.color_palette())
  result = postprocessor(outputs, torch.as_tensor(img.shape[-2:]).unsqueeze(0))[0]
  # The segmentation is stored in a special-format png
  panoptic_seg = Image.open(io.BytesIO(result['png_string']))
  panoptic_seg = np.array(panoptic_seg, dtype=np.uint8).copy()
  # We retrieve the ids corresponding to each mask
  panoptic_seg_id = rgb2id(panoptic_seg)

  # Finally we color each mask individually
  panoptic_seg[:, :, :] = 0
  for id in range(panoptic_seg_id.max() + 1):
    panoptic_seg[panoptic_seg_id == id] = np.asarray(next(palette)) * 255

  for threshold in [0.91]:
    
    probas_to_keep, bboxes_scaled = filter_bboxes_from_outputs(outputs,
                                                              threshold=threshold)

    
    
    plot_finetuned_results(my_image,
                           panoptic_seg,
                           probas_to_keep, 
                           bboxes_scaled,counter)

In [ ]:
def filter_bboxes_from_outputs(outputs,
                               threshold=0.7):
  
  # keep only predictions with confidence above threshold
  probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
  keep = probas.max(-1).values > threshold

  probas_to_keep = probas[keep]

  # convert boxes from [0; 1] to image scales
  bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
  
  return probas_to_keep, bboxes_scaled

In [ ]:
finetuned_classes =['rebar','spall','crack','stuff']

COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]
colors = COLORS *2     

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/

/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3


In [ ]:
import torch
import torch.nn.functional as F
from torch import nn


In [ ]:
#PATH = '/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/part2_output/checkpoint.pth'
PATH = '/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/segmentation_output/checkpoint.pth'
model,postprocessor = torch.hub.load('facebookresearch/detr', 'detr_resnet50_panoptic', pretrained=False,return_postprocessor=True,num_classes=4)

#from detr import (build)

checkpoint = torch.load(PATH,map_location='cpu',)
#/content/drive/MyDrive/Colab Notebooks/Session_14/detr/outputs
#/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/output_2
model.load_state_dict(checkpoint['model'],strict=True)
my_args = vars(checkpoint['args'])
my_args['device'] = 'cpu'
#my_args = argparse.Namespace(**my_args)
#build_op = detr.build(my_args)
model.eval();

Downloading: "https://github.com/facebookresearch/detr/archive/main.zip" to /root/.cache/torch/hub/main.zip
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/images

/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/images


In [ ]:
from PIL import Image

from os import listdir
counter = 1

imagesList = listdir('/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/coco_panoptic/panoptic_train2017/')
for images in imagesList:
  #img_name = '/content/drive/MyDrive/Colab Notebooks/Session_14/data/custom/train2017/14666848163_8be8e37562_k.jpg'
  #img_name = '/content/drive/MyDrive/Colab Notebooks/Session_14/data/custom/Test_baloon.jpg'
  im = Image.open(f'/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/coco/images/{images[:-4]}.jpg') #30-5.jpg ,0103, 0012,12-9,00026,001134,
  try:
    gt = Image.open(f'/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/coco_panoptic/panoptic_train2017/{images}')
  except:
    gt = Image.open(f'/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/coco_panoptic/panoptic_val2017/{images[:-4]}.png')
  #000136,12-9,0012,0036901
  # Difficult img  -- 0036901
  #im = Image.open(img_name)
  plt.figure(figsize=(20,20))
  plt.subplot(1, 4, 1)
  plt.imshow(im)
  plt.axis('off')
  plt.subplot(1, 4, 2)
  plt.imshow(gt)
  plt.axis('off')
  
  run_worflow(im,model,counter)
  
  plt.savefig(f'capstone_final_images{counter}.png')
  plt.show()
  
  
  if counter > 70:
    break
  counter = counter + 1


Output hidden; open in https://colab.research.google.com to view.